# Muse EEG

In this notebook we will train a model to associate EEG signals from the Muse 2 headset with the wearer's eyes being open or closed.

## Running a Survey

First we can import our library and create a survey, so we can train a model on the resulting data. We'll ask the participant to first get into a comfortable position, then open eyes for 30s, close for 30s, etc.

In [9]:
%matplotlib notebook
# Reload external source files when they change
%load_ext autoreload
%autoreload 2
import sys
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../src")
from recorder import Muse2EEGRecorder
from survey import Survey

eyes_open_step = (timedelta(seconds=30), "eyes_open", "Please open your eyes.", True)
eyes_closed_step = (timedelta(seconds=30), "eyes_closed", "Please close your eyes.", True)
eyes_schedule = [
    (timedelta(seconds=30), "intro", "Just breathe normally, gently relax any tension, get in a comfortable position.", False),
    eyes_open_step,
    eyes_closed_step,
    eyes_open_step,
    eyes_closed_step,
    eyes_open_step,
    eyes_closed_step
]

#eyes_survey = Survey(muse2_recorder, "Eyes open-closed", "Eyes open for 30, closed for 30 - repeat 3x.", eyes_schedule)
#eyes_survey.record("Jared")

Next step -- Just breathe normally, gently relax any tension, get in a comfortable position.
Next step -- Just breathe normally, gently relax any tension, get in a comfortable position.
Next step -- Survey Completed. Thank you.


'../data/muse2-recordings/surveys/TEST Eyes open-closed Jared 2021-06-24 15:46:45.509089'

## Preparing Data for Learning

We need to transform our raw survey data into a format suitable for supervised learning. We will create an input tensor with the shape required by PyTorch - `(batch_size, kernel_size, seq_len)`, aka `(Samples, Variables, Length / time or sequence steps)`, or `[batch_size, channels, num_features (aka: H * W)]`.

1. Batch size can be tuned. We will start with `64`.
2. The second index is the number of features per batch. In this case, we have four EEG sensors, so that will be `4`.
3. The number of samples included for each feature in each batch.

The PyTorch `DataLoader` is an alternative to batching out data manually. After creating a `Dataset`, the DataLoader will batch data with a given batch size.

In [214]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

# Load EEG CSV
eyes_closed_1 = pd.read_csv("../data/muse2-recordings/surveys/Eyes open-closed 2021-06-24 13:13:38.263066/eeg_raw-2_eyes_closed.csv")
eyes_open_1 = pd.read_csv("../data/muse2-recordings/surveys/Eyes open-closed 2021-06-24 13:13:38.263066/eeg_raw-3_eyes_open.csv")
print("Eyes closed shape:", eyes_closed_1.shape)
print("Eyes open shape:", eyes_open_1.shape)

# Create features ndarray
closed_x = eyes_closed_1[["eeg1", "eeg2", "eeg3", "eeg4"]].to_numpy()
open_x = eyes_open_1[["eeg1", "eeg2", "eeg3", "eeg4"]].to_numpy()
X = np.concatenate((closed_x, open_x))
print("Input features shape:", X.shape)
# TODO Scale data??

# Create one-hot labels ndarray
Y = np.vstack((
    # Eyes are open column
    np.concatenate((np.zeros((eyes_closed_1.shape[0])), np.ones((eyes_open_1.shape[0])))),
    # Eyes are closed column
    np.concatenate((np.ones((eyes_open_1.shape[0])), np.zeros((eyes_closed_1.shape[0]))))
)).T
print("Input labels shape:", Y.shape)

# Split into train, test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
print("Training data shape (features, labels):", (X_train.shape, Y_train.shape))

# Define a generic Dataset subclass
class OLDEEGDataset(Dataset):
    def __init__(self, data, labels, num_features, transform=None, target_transform=None):
        self.data = data
        self.labels = labels
        self.num_features = num_features
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        datum = self.data[idx]
        label = self.labels[idx]
        datum = np.expand_dims(datum, axis=0)
        if self.transform:
            datum = self.transform(datum)
        if self.target_transform:
            label = self.target_transform(label)
        return datum.T, label
    
# Define a generic Dataset subclass
class EEGDataset(Dataset):
    def __init__(self, data, labels, num_features, transform=None, target_transform=None):
        self.data = data
        self.labels = labels
        self.num_features = num_features
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return int(len(self.labels) / self.num_features)

    def __getitem__(self, idx):
        start_i = int(idx * self.num_features)
        end_i = start_i + self.num_features
        datum = self.data[start_i:end_i]
        #label = self.labels[start_i:end_i]
        label = self.labels[end_i]
        if self.transform:
            datum = self.transform(datum)
        if self.target_transform:
            label = self.target_transform(label)
        return datum.T, label

# Create PyTorch Datasets
train_dataset, test_dataset = EEGDataset(X_train, Y_train, 200), EEGDataset(X_test, Y_test, 200)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

Eyes closed shape: (7936, 7)
Eyes open shape: (7936, 7)
Input features shape: (15872, 4)
Input labels shape: (15872, 2)
Training data shape (features, labels): ((11904, 4), (11904, 2))


## PyTorch Model

Now we can create the neural network we will be training on the collected data. We will model it after the [simple BCI model by Sentdex](https://github.com/Sentdex/NNfSiX). Sentdex doesn't provide the training or validation data, so we will assume the input is simply batches of raw EEG data.

In [216]:
import torch
import torch.nn as nn
import torch.nn.functional as F

hidden_layers_size = 64
n_outputs = Y_train.shape[1]

# Create model
net = nn.Sequential(
    # Pass input to a 1D convolutional layer with a kernel size of 3, apply to activation function.
    nn.Conv1d(num_channels, hidden_layers_size, 3),
    nn.ReLU(),

    # Pass previous layer output to a 1D convolutional layer with a kernel size of 2, apply to activation function,
    # and get the max value from each kernel.
    nn.Conv1d(hidden_layers_size, hidden_layers_size, 2),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2),

    # Pass previous layer output to a 1D convolutional layer with a kernel size of 2, apply to activation function,
    # and get the max value from each kernel. (same as previous layer)
    nn.Conv1d(hidden_layers_size, hidden_layers_size, 2),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2),

    # Flatten the convolutions ?
    nn.Flatten(),
    
    # ?
    nn.Linear(3072, 512),
    #nn.ReLU(),

    # ?
    nn.Linear(512, n_outputs),
    nn.Softmax()
)

for i, data in enumerate(train_dataloader, 0):
    features, labels = data
    print("Model input shape:", features.shape)
    out = net(features.float())
    print("Model output shape:", out.shape)
    break

Model input shape: torch.Size([59, 4, 200])
Model output shape: torch.Size([59, 2])


### Training the model

In [232]:
from torch.utils.data import DataLoader
import torch.optim as optim

# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train for n epochs
n = 100
for epoch in range(n):
    running_loss = 0.
    for i, data in enumerate(train_dataloader, 0):
        features, labels = data

        # Zero gradients
        optimizer.zero_grad()

        # Forward
        print("Features shape:", features.shape)
        predictions = net(features.float())
        print("Predictions shape:", predictions.shape, "Labels shape:", labels.shape)
        
        # Convert labels from one-hot (n, 2) to categorical (n,)
        labels = labels[:, 0].long()
        # Get loss
        loss = criterion(predictions, labels.long())
        # Backward
        loss.backward()
        # Optimize
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 5 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels s

Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels shape: torch.Size([59, 2])
Features shape: torch.Size([59, 4, 200])
Predictions shape: torch.Size([59, 2]) Labels s

In [243]:
net.eval()
total = 0
correct = 0
for i, data in enumerate(tr_dataloader, 0):
    features, labels = data
    out = net(features.float())
    preds = F.log_softmax(out, dim=1).argmax(dim=1)
    print(labels.size(), preds.size())
    total += labels.size(0)
    correct += (preds == labels[:, 0]).sum().item()

print("Correct:", correct, "/", total)

torch.Size([59, 2]) torch.Size([59])
Correct: 41 / 59
